In [ ]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser, JsonOutputParser, XMLOutputParser, BaseOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough, ConfigurableField, chain
from langchain_core.exceptions import OutputParserException
from langchain_core.prompts import ChatPromptTemplate
from langchain.runnables.hub import HubRunnable
from pydantic import BaseModel, Field, model_validator
from typing import List, Iterator
from pprint import pprint
import json
from operator import itemgetter

# base_url = "https://api.deepseek.com/"

# model="deepseek-chat"
# llm = ChatOpenAI(model=model, temperature=0, api_key=api_key, base_url=base_url)

base_url="https://ark.cn-beijing.volces.com/api/v3"

model="doubao-1.5-pro-32k-250115"
llm = ChatOpenAI(model=model, temperature=0, api_key=api_key, base_url=base_url).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temprature of the LLM",
    )
)

# base_url="http://localhost:11434"
# model="deepseek-r1"
# llm = ChatOllama(model=model, temperature=0, base_url=base_url)

In [5]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
parser = StrOutputParser()
chain = prompt | llm | parser
async for chunk in chain.astream({"topic": "parrot"}):
    print(chunk, end="|", flush=True)

Here| is| a| funny| joke| about| a| par|rot|:|

|A| man| walks| into| a| pet| store| and| asks| to| see| the| talking| parro|ts|.| The| store| owner| shows| him| two| beautiful| birds|.|

|"|Here|,| this| one| is| $|5|,|0|0|0| and| the| other| is| $|1|0|,|0|0|0|,"| says| the| owner|.|

|The| customer| is| shocked| and| asks|,| "|Why| is| the| more| expensive| one| twice| the| price| of| the| other|?| Do| they| talk| differently|?"|

|The| owner| replies|,| "|No|,| they| both| talk| equally| well|.| The| $|1|0|,|0|0|0| par|rot| has| better| management| skills|.| The| $|5|,|0|0|0| par|rot| just| says|,| '|I|'m| a| par|rot|,'| but| the| $|1|0|,|0|0|0| par|rot| stands| up|,| looks| around|,| and| says|,| '|Well|,| it| looks| like| we|'re| all| out| of| parro|ts| today|!|'"| ||

In [6]:
chain = (
    llm | JsonOutputParser()
)
async for text in chain.astream(
    "以JSON格式输出法国，西班牙和日本的国家列表和人口数量。"
    '使用一个字典，包含外层键名为"countries"的国家列。'
    '每个国家应该有键名"name"和"population"'
):
    print(text, flush=True)

{}
{'countries': []}
{'countries': [{}]}
{'countries': [{'name': ''}]}
{'countries': [{'name': '法国'}]}
{'countries': [{'name': '法国', 'population': 6}]}
{'countries': [{'name': '法国', 'population': 67}]}
{'countries': [{'name': '法国', 'population': 678}]}
{'countries': [{'name': '法国', 'population': 6781}]}
{'countries': [{'name': '法国', 'population': 67813}]}
{'countries': [{'name': '法国', 'population': 678133}]}
{'countries': [{'name': '法国', 'population': 6781339}]}
{'countries': [{'name': '法国', 'population': 67813396}]}
{'countries': [{'name': '法国', 'population': 67813396}, {}]}
{'countries': [{'name': '法国', 'population': 67813396}, {'name': ''}]}
{'countries': [{'name': '法国', 'population': 67813396}, {'name': '西班牙'}]}
{'countries': [{'name': '法国', 'population': 67813396}, {'name': '西班牙', 'population': 4}]}
{'countries': [{'name': '法国', 'population': 67813396}, {'name': '西班牙', 'population': 47}]}
{'countries': [{'name': '法国', 'population': 67813396}, {'name': '西班牙', 'population': 474}]}
{

In [22]:
events = []
async for event in chain.astream_events("hello"):
    print(event["event"], flush=True)
    events.append(event)
print(f"Total {len(events)} events")

on_chain_start
on_chat_model_start
on_chat_model_stream
on_parser_start
on_chat_model_stream
on_chat_model_stream
on_chat_model_stream
on_chat_model_stream
on_chat_model_stream
on_chat_model_stream
on_chat_model_stream
on_chat_model_stream
on_chat_model_stream
on_chat_model_end
on_parser_end
on_chain_end
Total 16 events


In [31]:
joke_chain = ChatPromptTemplate.from_template("给我讲一个关于{topic}的笑话") | llm
poem_chain = (
    ChatPromptTemplate.from_template("给我写一首关于{topic}的绝句") | llm
)
main_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)
main_chain.get_graph().print_ascii()
main_chain.get_prompts()
main_chain.invoke({"topic": "程序员"})

               +--------------------------+                
               | Parallel<joke,poem>Input |                
               +--------------------------+                
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+*                     +------------+     
                   ***               ***                   
                      ***         ***   

[ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='给我讲一个关于{topic}的笑话'), additional_kwargs={})]),
 ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='给我写一首关于{topic}的绝句'), additional_kwargs={})])]

In [36]:
prompt1 = ChatPromptTemplate.from_template("给我讲一个关于{topic}的笑话") 
prompt2 = ChatPromptTemplate.from_template("这个笑话的主题是什么：{joke}") 

@chain
def custom_chain(text):
    prompt_val1 = prompt1.invoke({"topic": text})
    output1 = llm.invoke(prompt_val1)
    parsed_output1 = StrOutputParser().invoke(output1)

    chain2 = prompt2 | llm | StrOutputParser()

    return chain2.invoke({"joke": parsed_output1})

custom_chain.invoke("熊")

'这两个笑话的主题都围绕着熊和毛发展开，主要体现出一种荒诞、意外的幽默风格。\n\n### “北极熊拔毛”笑话主题\n - 该笑话主题侧重于展现一种无厘头的行为和意外的结果。北极熊出于无聊进行拔毛这一荒诞行为，当它把毛拔光后才感受到寒冷，这种行为与结果之间的逻辑反差产生了幽默效果，主题核心是凸显这种无意义行为带来的意外后果，以荒诞情景引人发笑。\n\n### “熊和兔子”笑话主题\n - 此笑话主题同样是利用意外和荒诞元素制造笑点。熊询问兔子是否掉毛，看似是一个正常的交流，但它询问的目的却是抓起兔子擦屁股，这一意外的发展打破了常规的思维逻辑，兔子的回答和熊的实际行为之间形成了强烈反差，主题在于通过这种意外情节来营造幽默氛围。 '

In [44]:
def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1)*len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")} | RunnableLambda(multiple_length_function)
    }
    | prompt | llm
)
chain.invoke({"foo": "bar", "bar": "gah3"})


AIMessage(content='To find the sum of 3 and 12, you simply add the two numbers together.\n\n3 + 12 = 15\n\nSo, the result is 15. ', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 16, 'total_tokens': 57, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'doubao-1-5-pro-32k-250115', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--117fc9a2-14f6-4d89-adc7-3273291f6387-0', usage_metadata={'input_tokens': 16, 'output_tokens': 41, 'total_tokens': 57, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [59]:
prompt = ChatPromptTemplate.from_template("请列出5个与以下动物相似的动物名称，用逗号分隔：{animal}。不要包含数字")
str_chain = prompt | llm | StrOutputParser()
for chunk in str_chain.stream({"animal": "熊"}):
    print(chunk, end="", flush=True)
print()

def split_into_list(input: Iterator[str]) -> Iterator[List[str]]:
    buffer = ""
    for chunk in input:
        buffer += chunk;
        while "，" in buffer:
            comma_index = buffer.index("，")
            yield [buffer[:comma_index].strip()]
            buffer = buffer[comma_index+1:]
    yield [buffer.strip()]

list_chain = str_chain | split_into_list
for chunk in list_chain.stream({"animal": "熊"}):
    print(chunk, end="\n", flush=True)

熊猫，棕熊，北极熊，浣熊，貂熊
['熊猫']
['北极熊']
['棕熊']
['黑熊']
['马来熊']


In [56]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x:x["num"]+1
)
runnable.invoke({"num":1})

{'passed': {'num': 1}, 'modified': 2}

In [10]:
# llm.invoke("请给我一个哺乳动物的名称")
llm.with_config(configurable={"llm_temperature": 0.9}).invoke("请给我一个哺乳动物的名称")

AIMessage(content='以下为你列举不同类型的哺乳动物名称：\n### 陆地哺乳动物\n- **老虎**：老虎是世界上最大的猫科动物之一，也是顶级的食肉动物。它拥有强壮的身体、锋利的爪子和牙齿，捕猎能力极强。老虎主要分布在亚洲的部分地区，由于栖息地破坏和偷猎等原因，许多老虎种类都面临濒危的困境。\n- **大象**：大象是陆地上最大的哺乳动物，拥有庞大而强壮的身体，长长的鼻子（象鼻）非常灵活，可用于抓取物体、喝水等。大象是草食性动物，主要分布在非洲和亚洲的一些地区。\n- **大熊猫**：大熊猫是中国特有的珍稀动物，憨态可掬，深受人们喜爱。它们主要以竹子为食，有着圆滚滚的身体和黑白相间的毛色，其栖息地主要在中国四川、陕西和甘肃等地的山区。\n### 海洋哺乳动物\n- **海豚**：海豚是一种非常聪明且可爱的海洋哺乳动物。它们具有流线型的身体，游泳速度快，生活在世界各大洋的温暖海域。海豚喜欢群居，通过发出各种声音进行交流，还能表演各种高难度的动作。\n- **鲸鱼**：鲸鱼是海洋中的巨型生物，其中蓝鲸是地球上现存体型最大的动物。鲸鱼用肺呼吸，需要定期浮出水面换气。不同种类的鲸鱼食性有所不同，有的主要以小鱼、小虾为食，有的则捕食大型海洋生物。\n### 空中哺乳动物\n- **蝙蝠**：蝙蝠是唯一能够真正飞行的哺乳动物。它们的前肢进化成了翅膀，通常在夜间活动，依靠回声定位来捕食昆虫、水果等食物。蝙蝠种类繁多，广泛分布于世界各地。 ', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 366, 'prompt_tokens': 15, 'total_tokens': 381, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_n

In [17]:
prompt = HubRunnable("rlm/rag-prompt").configurable_fields(
    owner_repo_commit=ConfigurableField(
        id="hub_commit",
        name="Hub Commit",
        description="The Hub commit to pull from",
    )
)
prompt.invoke({"question": "bar", "context": "gah3"})
# prompt.with_config(configurable={"hub_commit": "rlm/rag-prompt-llama"}).invoke({"question": "bar", "context": "gah3"})

/Users/bytedance/miniconda3/envs/langchain/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: bar \nContext: gah3 \nAnswer:", additional_kwargs={}, response_metadata={})])